Script serves two purposes:
Curates data for 3d grader
Builds models for grading EiPE answers (Data is 3D)

In [87]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.mixture import GaussianMixture
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold


from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier


import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay


nltk.download('punkt')
nltk.download('words')

from nltk.corpus import words




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chine\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\chine\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [88]:
data_file = "sigcse_2024.csv"
alldata_df = pd.read_csv(data_file, encoding="utf-8")


In [89]:
alldata_df.head()

,subset,embedding_subset,embedding_index,pl_qid,qid,code,assumption,example_correct_answers,response,binary_annotators,...,chinny_una,chinny_c,chinny_hl,binglin_una,binglin_c,binglin_hl,una,c,hl,3d_resolution
0,train,train,97,cdrd_exam1_manual/cdrd_exam1_1,print_found_if_three_numbers_unique,"def f(x, y, z):\n if x != y and x != z and ...","Assume that the variables x, y, and z are inte...",['Print the word found if all three numbers ar...,the code checks if all arguments regarding var...,"['Austin', 'Max']",...,0,0,0.0,0,0,0,1,0,0,reconcile
1,validate,train,113,cdrd_exam1_manual/cdrd_exam1_1,print_items_out_smaller_then_larger,"def f(x, y):\n if x < y:\n print(x, ...",Assume that the variables x and y are integers.,['prints two given numbers in numberical order...,prints two numbers ordered smallest to largest,"['Austin', 'Max']",...,1,1,1.0,1,1,1,0,1,1,reconcile
2,train,train,138,cdrd_exam1_manual/cdrd_exam1_2,computes_average_of_list,def f(x):\n return sum(x) / len(x),Assume that the variable x is a non-empty list...,"['Returns the average value in a list', 'finds...",returns the average of a list,"['Binglin', 'Rachel']",...,1,1,1.0,1,1,1,1,1,1,reconcile
3,validate,train,150,cdrd_week5_exam,does_list_have_value_above_y,"def f(x, y):\n for val in x:\n if va...",Assume that the variable x is a list of intege...,['Return True if at least one list element is ...,returns whether the first value in a list is g...,['Binglin'],...,1,0,1.0,1,0,1,1,0,1,reconcile
4,train,train,228,cdrd_exam3_manual/cdrd_exam3_1,count how many times a given string appears in...,"def f(x, y):\n k = 0\n for e in x:\n ...",Assume that variable x is a list of strings an...,['Return a count of how many times a given str...,return the number of times in which the string...,"['Binglin', 'Rachel']",...,1,1,0.0,1,1,1,1,1,1,reconcile


In [90]:
def display_class_dist(given_df):
    '''
    Display the distribution of data for each qid. There are 8 possible bins.
    '''
    classes =  {(0,0,0): 0, (0,0,1):0, (0,1,0):0, (0,1,1):0, 
                (1,0,0):0, (1,1,0):0, (1,0,1):0, (1,1,1):0}
    
    ordering = [(0,0,0), (0,0,1), (0,1,0), (0,1,1), (1,0,0), (1,1,0), (1,0,1), (1,1,1)]
    
    print("3D groupings = (unambig, correct, highlevel)\n\n")
    print("3D label,  Freq. (Num Times Occur)")
    
    numrowsdata = 0
    
    for index, row in given_df.iterrows():
        
        cur_unambig = row["una"]
        cur_correct = row["c"]
        cur_hl      = row["hl"]
        
        classes[(cur_unambig, cur_correct, cur_hl)] += 1
        numrowsdata += 1
    
    for curbin in ordering:
        curbincount = classes[curbin] #get the number of occurrences of a particular labeling configuration within the data
        print("{}  {:>5.2f}% ({})".format(curbin, round(100*curbincount/numrowsdata, 2), curbincount)) #print group name, group freq and raw count of group occur.


    
def qid_dist(given_df):
    """
    Show how frequently each qid occured in the data
    """
    pass

In [91]:
filter1 = alldata_df[alldata_df["subset"] == "train"]
print(filter1.shape)
filter2= filter1[filter1["qid"] == "computes_average_of_list"]
print(filter2.shape)
display_class_dist(filter2)

display_class_dist(alldata_df[alldata_df["qid"] == "computes_average_of_list"])

(1832, 24)
(72, 24)
3D groupings = (unambig, correct, highlevel)


3D label,  Freq. (Num Times Occur)
(0, 0, 0)   0.00% (0)
(0, 0, 1)   0.00% (0)
(0, 1, 0)   0.00% (0)
(0, 1, 1)   8.33% (6)
(1, 0, 0)   0.00% (0)
(1, 1, 0)  44.44% (32)
(1, 0, 1)   0.00% (0)
(1, 1, 1)  47.22% (34)
3D groupings = (unambig, correct, highlevel)


3D label,  Freq. (Num Times Occur)
(0, 0, 0)   0.00% (0)
(0, 0, 1)   0.00% (0)
(0, 1, 0)   0.00% (0)
(0, 1, 1)   9.17% (11)
(1, 0, 0)   0.83% (1)
(1, 1, 0)  42.50% (51)
(1, 0, 1)   0.00% (0)
(1, 1, 1)  47.50% (57)


In [92]:
#distribution of all the data, even those tuples that were manually reconciled.
display_class_dist(alldata_df)

3D groupings = (unambig, correct, highlevel)


3D label,  Freq. (Num Times Occur)
(0, 0, 0)   6.01% (184)
(0, 0, 1)  13.38% (410)
(0, 1, 0)   1.44% (44)
(0, 1, 1)   9.76% (299)
(1, 0, 0)   4.44% (136)
(1, 1, 0)   6.59% (202)
(1, 0, 1)  20.30% (622)
(1, 1, 1)  38.09% (1167)


In [93]:
stemmer = PorterStemmer()

def preprocess_text(studentresponse):    
    words = word_tokenize(studentresponse)
    stemmed_words = [stemmer.stem(word) for word in words]
    lower_words= [word.lower() for word in stemmed_words] #convert to lowercase
    stemmed_words = lower_words
    regular_alpha = [] 
    for word in stemmed_words:
        if word.isalnum():  #remove any word that isn't an alphabet or a number.
            regular_alpha.append(word)
    stemmed_words = regular_alpha
    
    #TODO - need to handle stop words by passing in my own list. (related to tf-idf weighting)
    
    return " ".join(stemmed_words)

In [94]:
#distance metrics
#TODO: consider a few metrics


def cosine_similarity(x, y):
    #compute the cosine similarity of two vectors

    return sklearn.metrics.pairwise.cosine_similarity(x, y).item()     #using .item() to get value as a scalar
    
def distance_from_golden(golden, given_ans, dist_func):
    #compute a distance metric for the distance between a given answer and the golden answer
    distance = dist_func(golden, given_ans)
    if distance == None:
        return float("inf")
    else:
        return distance

In [147]:
def plot_confusion_matrix(confusion_matrix, what_to_predict, accuracy ):
        '''
        Function to help with plotting a confusion matrix
        '''
        plt.figure(figsize=(9,9))
        sns.heatmap(confusion_matrix, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
        plt.ylabel('Actual label');
        plt.xlabel('Predicted label');
        all_sample_title = 'Accuracy Score for {thingtopredict}: {accscore:.2f}'.format(thingtopredict = what_to_predict,
                                                                                    accscore = accuracy)
        plt.title(all_sample_title, size = 15);
        

def process_raw_features(given_df, pct_for_train = 0.8):
    """
    Process the raw text and get it ready to be trained
    Return processed text as a list of lists, where each list is one student response or one row of data
    """
    
    numsamples = 0
    limit = int(pct_for_train * len(given_df["response"]))
    processed_studentanswers = []
    
    for studentanswer in given_df["response"]:
        
        processed_answer = preprocess_text(studentanswer)
        processed_studentanswers.append(processed_answer)
        numsamples += 1
        
        if numsamples == limit:
            break
    return processed_studentanswers


def add_similarity_vector(golden_answers, students_responses_as_vec, curvec_fitter):
    """
    Create a column vector that represents the distance of a given student answer from the golden answer
    """
    golden_answers_vectors = curvec_fitter.transform(golden_answers).toarray()
    
    sim_vector = []
    
    print("Shape of all student vectors = {}".format(students_responses_as_vec.shape))
    print("Shape of golden_answers_vectors = {}".format(golden_answers_vectors.shape))
    
    for curstudent_vec in students_responses_as_vec:
        
        print("Shape single student vector = {}".format(curstudent_vec.shape))
        best_similar = float("-inf") #ranges from 0 to 1 (1 meaning very similar or identical)
        
        for single_golden_answer_vec in golden_answers_vectors:
            print("Shape single golden answer vector = {}".format(single_golden_answer_vec.shape))
            #cur_distance = distance_from_golden(single_golden_answer_vec, curstudent_vec, cosine_similarity)
            #compute distance. reshape the data into row vectors
            
            parta = single_golden_answer_vec.reshape(1,-1)
            partb = curstudent_vec.reshape(1,-1)
            
            print("After reshaping, current size = {} for golden answer and  {} for student vector".format(parta.shape, partb.shape))
            
            cur_distance = cosine_similarity(parta, partb)
            best_similar = max(best_similar, cur_distance)
            
        sim_vector.append(best_similar)
    
    sim_vector = np.array(sim_vector)
    sim_vector = sim_vector.reshape(-1,1) #need to convert into a proper column vector
    return sim_vector


def train_model_and_validate(given_train_df, given_validate_df, golden_answers = [], qname = ""):
    
    ydims =  ["una", "c", "hl"]
    
    
    X_train_processed_text = process_raw_features(given_train_df, pct_for_train = 1)
 
    vectorizer = CountVectorizer(ngram_range = (2,2), min_df = 2)
    vec_fitter = vectorizer.fit(X_train_processed_text)    #fit only on training data to prevent overfit
    
    
    X_train = vec_fitter.transform(X_train_processed_text).toarray() #transform data into vectors and convert it to array    
    y_train = given_train_df[ydims].to_numpy(dtype=int)
    
    X_validate_processed_text = process_raw_features(given_validate_df, pct_for_train = 1)
    X_validate = vec_fitter.transform(X_validate_processed_text).toarray()  #use the previously fitted vectorizer to maintain shape
    y_validate = given_validate_df[ydims].to_numpy(dtype=int)
    
    if len(golden_answers) > 0:
    
        distance_from_golden_vector_trainset = add_similarity_vector(golden_answers, X_train, vec_fitter)
        distance_from_golden_vector_validationset  = add_similarity_vector(golden_answers, X_validate, vec_fitter)
        
        print("Original X data shape = {}, golden_answer_shape = {}".format(X_train.shape,
                                                                           distance_from_golden_vector_trainset.shape))
        
        X_train = np.append(X_train, distance_from_golden_vector_trainset, axis = 1)
        X_validate = np.append(X_validate, distance_from_golden_vector_validationset, axis = 1)
        
#         X_train.append(distance_from_golden_vector_trainset, axis = 1)
#         X_validate.append(distance_from_golden_vector_validationset, axis = 1)
    
    
    
 
        
    labels_order = ["Unambig", "Correct", "High level"]
    accuracy_l = []
    
    

    for i in range(3):
        model = LogisticRegression(random_state = 0)
        
        num_classes = list(np.unique(y_train[:,i])) #check number of classes in data. should be two
        print("We have {} classes".format(len(num_classes)))
        if len(num_classes) == 1: #change the last y in the tuple to force training to go through - otherwise, throwing error
            y_train[-1, i] = 0 if y_train[-1, i] == 1 else 0
        
        model = model.fit(X_train, y_train[:,i]) #train only one column at a time
        
        binary_y_predictions = model.predict(X_validate)
        
        #use the withheld set for testing the model
        print("{} Log reg. model performance on the withheld test set:\n\n".format(labels_order[i]))
        
        #for the y_test data (the true labels for column i), check if the predictions match
        dim_accuracy = metrics.accuracy_score(y_validate[:, i], binary_y_predictions)
        print("Accuracy is = ", dim_accuracy)
        accuracy_l.append(dim_accuracy)
        #print(metrics.classification_report(y_test, binary_y_predictions))


        print("\n\nConfusion Matrix:")
        
        confusion_matrix = metrics.confusion_matrix(y_validate[:, i],binary_y_predictions, normalize="true")
        print(confusion_matrix)
        
        #uncomment next line to print the confusion matrix
        #plot_confusion_matrix(confusion_matrix, labels_order[i] + qname,  dim_accuracy)
        print("\n\n")
        
    return accuracy_l  #accuracy for the three dimensions for a given problem


In [145]:
#for each question, create the input representation
#train various models, bigrams, bigrams + distance_from_golden, report on performance

train_pct = 1 #num from 0 to 1. currently 1 because want to train with 100% of train dataset
qids_accuracy = []

train_df = alldata_df[alldata_df["subset"] == "train"]       #select only training rows
validate_df = alldata_df[alldata_df["subset"] == "validate"] #only validation row

#alldata_df[alldata_df["subset"] == "train" & alldata_df["qid"] == "computes average of list"]


for cur_qid in qids:
    
    cur_question_training_df = train_df[train_df["qid"] == cur_qid]
    cur_question_validation_df = validate_df[validate_df["qid"] == cur_qid]
    
    print("QID = \'{}.\'".format(cur_qid))
    qname_formatted = ": " + cur_qid #format cur_qid name so it looks nice when printed with other function
    
    cur_qid_accuracy = train_model_and_validate(cur_question_training_df, cur_question_validation_df, qname = qname_formatted)
    
    qids_accuracy.append(cur_qid_accuracy)
    print("\n\n")


  

QID = 'print_found_if_three_numbers_unique.'
We have 2 classes
Unambig Log reg. model performance on the withheld test set:


Accuracy is =  0.5625


Confusion Matrix:
[[0.23529412 0.76470588]
 [0.06666667 0.93333333]]



We have 2 classes
Correct Log reg. model performance on the withheld test set:


Accuracy is =  0.8125


Confusion Matrix:
[[0.33333333 0.66666667]
 [0.07692308 0.92307692]]



We have 2 classes
High level Log reg. model performance on the withheld test set:


Accuracy is =  0.875


Confusion Matrix:
[[0.2 0.8]
 [0.  1. ]]






QID = 'print_items_out_smaller_then_larger.'
We have 2 classes
Unambig Log reg. model performance on the withheld test set:


Accuracy is =  0.8144329896907216


Confusion Matrix:
[[0.42857143 0.57142857]
 [0.02898551 0.97101449]]



We have 2 classes
Correct Log reg. model performance on the withheld test set:


Accuracy is =  0.8144329896907216


Confusion Matrix:
[[0.53846154 0.46153846]
 [0.08450704 0.91549296]]



We have 2 classes
High l

High level Log reg. model performance on the withheld test set:


Accuracy is =  0.7727272727272727


Confusion Matrix:
[[0.57142857 0.42857143]
 [0.13333333 0.86666667]]






QID = 'absolute_value.'
We have 2 classes
Unambig Log reg. model performance on the withheld test set:


Accuracy is =  0.6


Confusion Matrix:
[[0.375      0.625     ]
 [0.14285714 0.85714286]]



We have 2 classes
Correct Log reg. model performance on the withheld test set:


Accuracy is =  0.7333333333333333


Confusion Matrix:
[[1.         0.        ]
 [0.44444444 0.55555556]]



We have 2 classes
High level Log reg. model performance on the withheld test set:


Accuracy is =  0.8


Confusion Matrix:
[[0.4 0.6]
 [0.  1. ]]






QID = 'is_even.'
We have 2 classes
Unambig Log reg. model performance on the withheld test set:


Accuracy is =  0.7142857142857143


Confusion Matrix:
[[0.         1.        ]
 [0.09090909 0.90909091]]



We have 2 classes
Correct Log reg. model performance on the withheld test set:

In [148]:
#2nd model which incorporates the distance to the closest golden answer 

import ast
        

for cur_qid in qids:
    
        
    cur_question_training_df = train_df[train_df["qid"] == cur_qid]
    cur_question_validation_df = validate_df[validate_df["qid"] == cur_qid]
    
    print("QID = \'{}.\'".format(cur_qid))
    qname_formatted = ": " + cur_qid #format cur_qid name so it looks nice when printed with other function
    
    q_goldenanswers = cur_question_training_df["example_correct_answers"].iloc[0]
    q_goldenanswers = ast.literal_eval(q_goldenanswers)
    
    if type(q_goldenanswers) != list:
        print("Had to convert to list")
        q_goldenanswers = [q_goldenanswers]

    
    cur_qid_accuracy = train_model_and_validate(cur_question_training_df, cur_question_validation_df, 
                                                qname = qname_formatted, golden_answers = q_goldenanswers)
    
    qids_accuracy.append(cur_qid_accuracy)
    print("\n\n")


QID = 'print_found_if_three_numbers_unique.'
Shape of all student vectors = (96, 116)
Shape of golden_answers_vectors = (2, 116)
Shape single student vector = (116,)
Shape single golden answer vector = (116,)
After reshaping, current size = (1, 116) for golden answer and  (1, 116) for student vector
Shape single golden answer vector = (116,)
After reshaping, current size = (1, 116) for golden answer and  (1, 116) for student vector
Shape single student vector = (116,)
Shape single golden answer vector = (116,)
After reshaping, current size = (1, 116) for golden answer and  (1, 116) for student vector
Shape single golden answer vector = (116,)
After reshaping, current size = (1, 116) for golden answer and  (1, 116) for student vector
Shape single student vector = (116,)
Shape single golden answer vector = (116,)
After reshaping, current size = (1, 116) for golden answer and  (1, 116) for student vector
Shape single golden answer vector = (116,)
After reshaping, current size = (1, 116) f

After reshaping, current size = (1, 351) for golden answer and  (1, 351) for student vector
Shape single golden answer vector = (351,)
After reshaping, current size = (1, 351) for golden answer and  (1, 351) for student vector
Shape single golden answer vector = (351,)
After reshaping, current size = (1, 351) for golden answer and  (1, 351) for student vector
Shape single student vector = (351,)
Shape single golden answer vector = (351,)
After reshaping, current size = (1, 351) for golden answer and  (1, 351) for student vector
Shape single golden answer vector = (351,)
After reshaping, current size = (1, 351) for golden answer and  (1, 351) for student vector
Shape single golden answer vector = (351,)
After reshaping, current size = (1, 351) for golden answer and  (1, 351) for student vector
Shape single student vector = (351,)
Shape single golden answer vector = (351,)
After reshaping, current size = (1, 351) for golden answer and  (1, 351) for student vector
Shape single golden answ

After reshaping, current size = (1, 351) for golden answer and  (1, 351) for student vector
Shape single golden answer vector = (351,)
After reshaping, current size = (1, 351) for golden answer and  (1, 351) for student vector
Shape single student vector = (351,)
Shape single golden answer vector = (351,)
After reshaping, current size = (1, 351) for golden answer and  (1, 351) for student vector
Shape single golden answer vector = (351,)
After reshaping, current size = (1, 351) for golden answer and  (1, 351) for student vector
Shape single golden answer vector = (351,)
After reshaping, current size = (1, 351) for golden answer and  (1, 351) for student vector
Shape single student vector = (351,)
Shape single golden answer vector = (351,)
After reshaping, current size = (1, 351) for golden answer and  (1, 351) for student vector
Shape single golden answer vector = (351,)
After reshaping, current size = (1, 351) for golden answer and  (1, 351) for student vector
Shape single golden answ

Shape of all student vectors = (45, 69)
Shape of golden_answers_vectors = (3, 69)
Shape single student vector = (69,)
Shape single golden answer vector = (69,)
After reshaping, current size = (1, 69) for golden answer and  (1, 69) for student vector
Shape single golden answer vector = (69,)
After reshaping, current size = (1, 69) for golden answer and  (1, 69) for student vector
Shape single golden answer vector = (69,)
After reshaping, current size = (1, 69) for golden answer and  (1, 69) for student vector
Shape single student vector = (69,)
Shape single golden answer vector = (69,)
After reshaping, current size = (1, 69) for golden answer and  (1, 69) for student vector
Shape single golden answer vector = (69,)
After reshaping, current size = (1, 69) for golden answer and  (1, 69) for student vector
Shape single golden answer vector = (69,)
After reshaping, current size = (1, 69) for golden answer and  (1, 69) for student vector
Shape single student vector = (69,)
Shape single golde

Shape single golden answer vector = (100,)
After reshaping, current size = (1, 100) for golden answer and  (1, 100) for student vector
Shape single student vector = (100,)
Shape single golden answer vector = (100,)
After reshaping, current size = (1, 100) for golden answer and  (1, 100) for student vector
Shape single golden answer vector = (100,)
After reshaping, current size = (1, 100) for golden answer and  (1, 100) for student vector
Shape single student vector = (100,)
Shape single golden answer vector = (100,)
After reshaping, current size = (1, 100) for golden answer and  (1, 100) for student vector
Shape single golden answer vector = (100,)
After reshaping, current size = (1, 100) for golden answer and  (1, 100) for student vector
Shape single student vector = (100,)
Shape single golden answer vector = (100,)
After reshaping, current size = (1, 100) for golden answer and  (1, 100) for student vector
Shape single golden answer vector = (100,)
After reshaping, current size = (1, 

...

Shape single golden answer vector = (188,)
After reshaping, current size = (1, 188) for golden answer and  (1, 188) for student vector
Shape single student vector = (188,)
Shape single golden answer vector = (188,)
After reshaping, current size = (1, 188) for golden answer and  (1, 188) for student vector
Shape single golden answer vector = (188,)
After reshaping, current size = (1, 188) for golden answer and  (1, 188) for student vector
Shape single golden answer vector = (188,)
After reshaping, current size = (1, 188) for golden answer and  (1, 188) for student vector
Shape single student vector = (188,)
Shape single golden answer vector = (188,)
After reshaping, current size = (1, 188) for golden answer and  (1, 188) for student vector
Shape single golden answer vector = (188,)
After reshaping, current size = (1, 188) for golden answer and  (1, 188) for student vector
Shape single golden answer vector = (188,)
After reshaping, current size = (1, 188) for golden answer and  (1, 188) 

Shape single golden answer vector = (188,)
After reshaping, current size = (1, 188) for golden answer and  (1, 188) for student vector
Shape single golden answer vector = (188,)
After reshaping, current size = (1, 188) for golden answer and  (1, 188) for student vector
Shape single student vector = (188,)
Shape single golden answer vector = (188,)
After reshaping, current size = (1, 188) for golden answer and  (1, 188) for student vector
Shape single golden answer vector = (188,)
After reshaping, current size = (1, 188) for golden answer and  (1, 188) for student vector
Shape single golden answer vector = (188,)
After reshaping, current size = (1, 188) for golden answer and  (1, 188) for student vector
Shape single student vector = (188,)
Shape single golden answer vector = (188,)
After reshaping, current size = (1, 188) for golden answer and  (1, 188) for student vector
Shape single golden answer vector = (188,)
After reshaping, current size = (1, 188) for golden answer and  (1, 188) 

Shape single golden answer vector = (127,)
After reshaping, current size = (1, 127) for golden answer and  (1, 127) for student vector
Shape single golden answer vector = (127,)
After reshaping, current size = (1, 127) for golden answer and  (1, 127) for student vector
Shape single student vector = (127,)
Shape single golden answer vector = (127,)
After reshaping, current size = (1, 127) for golden answer and  (1, 127) for student vector
Shape single golden answer vector = (127,)
After reshaping, current size = (1, 127) for golden answer and  (1, 127) for student vector
Shape single golden answer vector = (127,)
After reshaping, current size = (1, 127) for golden answer and  (1, 127) for student vector
Shape single student vector = (127,)
Shape single golden answer vector = (127,)
After reshaping, current size = (1, 127) for golden answer and  (1, 127) for student vector
Shape single golden answer vector = (127,)
After reshaping, current size = (1, 127) for golden answer and  (1, 127) 

Shape single golden answer vector = (133,)
After reshaping, current size = (1, 133) for golden answer and  (1, 133) for student vector
Shape single golden answer vector = (133,)
After reshaping, current size = (1, 133) for golden answer and  (1, 133) for student vector
Shape single student vector = (133,)
Shape single golden answer vector = (133,)
After reshaping, current size = (1, 133) for golden answer and  (1, 133) for student vector
Shape single golden answer vector = (133,)
After reshaping, current size = (1, 133) for golden answer and  (1, 133) for student vector
Shape single golden answer vector = (133,)
After reshaping, current size = (1, 133) for golden answer and  (1, 133) for student vector
Shape single student vector = (133,)
Shape single golden answer vector = (133,)
After reshaping, current size = (1, 133) for golden answer and  (1, 133) for student vector
Shape single golden answer vector = (133,)
After reshaping, current size = (1, 133) for golden answer and  (1, 133) 

We have 2 classes
Unambig Log reg. model performance on the withheld test set:


Accuracy is =  0.5909090909090909


Confusion Matrix:
[[0.33333333 0.66666667]
 [0.23076923 0.76923077]]



We have 2 classes
Correct Log reg. model performance on the withheld test set:


Accuracy is =  0.9090909090909091


Confusion Matrix:
[[0.91666667 0.08333333]
 [0.1        0.9       ]]



We have 2 classes
High level Log reg. model performance on the withheld test set:


Accuracy is =  0.7272727272727273


Confusion Matrix:
[[0.42857143 0.57142857]
 [0.13333333 0.86666667]]






QID = 'absolute_value.'
Shape of all student vectors = (45, 46)
Shape of golden_answers_vectors = (3, 46)
Shape single student vector = (46,)
Shape single golden answer vector = (46,)
After reshaping, current size = (1, 46) for golden answer and  (1, 46) for student vector
Shape single golden answer vector = (46,)
After reshaping, current size = (1, 46) for golden answer and  (1, 46) for student vector
Shape single golden a

In [149]:
#show the accuracy of the classifier for each column across all data



qids_accuracy = np.array(qids_accuracy)
print("Averaging results for all qids in each of the respective dimensions\n")

avg_unambig_accuracy = np.mean(qids_accuracy[:, 0])
print("Avg accuracy for Unambig: {:.2f}".format(avg_unambig_accuracy))


avg_corr_accuracy = np.mean(qids_accuracy[:, 1])
print("Avg accuracy for Correct column: {:.2f}".format(avg_corr_accuracy))



avg_highlvl_accuracy = np.mean(qids_accuracy[:, 2])
print("Avg accuracy for High-level column: {:.2f}\n\n".format(avg_highlvl_accuracy))


avg_classifier_perf_overall = np.mean(qids_accuracy)
print("Average results across all dimensions for all qids: {:.2f}".format(avg_classifier_perf_overall))

Averaging results for all qids in each of the respective dimensions

Avg accuracy for Unambig: 0.73
Avg accuracy for Correct column: 0.80
Avg accuracy for High-level column: 0.86


Average results across all dimensions for all qids: 0.80


In [187]:

def train_model_and_validate_specify_order(given_train_df, given_validate_df, golden_answers = [], given_dims_order = [], qname = ""):
    
    ydims = given_dims_order #["una", "c", "hl"]
    

    X_train_processed_text = process_raw_features(given_train_df, pct_for_train = 1)
 
    vectorizer = CountVectorizer(ngram_range = (2,2), min_df = 2)
    vec_fitter = vectorizer.fit(X_train_processed_text)    #fit only on training data to prevent overfit
    
    
    X_train = vec_fitter.transform(X_train_processed_text).toarray() #transform data into vectors and convert it to array    
    
    
    
    y_train = given_train_df[ydims].to_numpy(dtype=int)
    
    X_validate_processed_text = process_raw_features(given_validate_df, pct_for_train = 1)
    X_validate = vec_fitter.transform(X_validate_processed_text).toarray()  #use the previously fitted vectorizer to maintain shape
    y_validate = given_validate_df[ydims].to_numpy(dtype=int)
    
    if len(golden_answers) > 0:
    
        distance_from_golden_vector_trainset = add_similarity_vector(golden_answers, X_train, vec_fitter)
        distance_from_golden_vector_validationset  = add_similarity_vector(golden_answers, X_validate, vec_fitter)
        
        print("Original X data shape = {}, golden_answer_shape = {}".format(X_train.shape,
                                                                           distance_from_golden_vector_trainset.shape))
        
        X_train = np.append(X_train, distance_from_golden_vector_trainset, axis = 1)
        X_validate = np.append(X_validate, distance_from_golden_vector_validationset, axis = 1)
        
#         X_train.append(distance_from_golden_vector_trainset, axis = 1)
#         X_validate.append(distance_from_golden_vector_validationset, axis = 1)
    
    
    
        
    labels_order = ["Unambig", "Correct", "High level"]
    accuracy_l = []
  
    
    col1name, col2name, col3name = given_dims_order #order of the y dimensions
    
    y_firstcol_train_vector = given_train_df[col1name].to_numpy(dtype=int) 
    y_secondcol_train_vector = given_train_df[col2name].to_numpy(dtype=int)
    y_thirdcol_train_vector = given_train_df[col3name].to_numpy(dtype=int)
    
    
    #potential
    
    firstdim_feedback_indices = []
    seconddim_feedback_indices = []
    thirddim_feedback_indices = []
    
    
    num_classes = list(np.unique(y_firstcol_train_vector)) #check number of classes in data. should be two
    if len(num_classes) == 1: #change the last y in the tuple to force training to go through - otherwise, throwing error
        y_firstcol_train_vector[-1] = 0 if y_firstcol_train_vector[-1] == 1 else 0

        
    model1 = LogisticRegression(random_state = 0)
    model1 = model1.fit(X_train, y_firstcol_train_vector) #train only one column at a time
    round1_y_predictions = model1.predict(X_train)
    
    
    for i in range(len(round1_y_predictions)):
        if round1_y_predictions[i] == 1: #no feedback needed for that dimension. feedback potentially for next dimension
            firstdim_feedback_indices.append(i)
    
    
    round2_X_train_input = []
    round2_y_train_input = []
    for cur_index in firstdim_feedback_indices:
        round2_X_train_input.append(X_train[cur_index, :])
        round2_y_train_input.append(y_secondcol_train_vector[cur_index])
    
    
    round2_X_train_input = np.array(round2_X_train_input)
    round2_y_train_input = np.array(round2_y_train_input)
    
    model2 = LogisticRegression(random_state = 0)
    model2 = model2.fit(round2_X_train_input, round2_y_train_input)
    round2_y_predictions = model2.predict(round2_X_train_input)
    
    
    round3_X_train_input = []
    round3_y_train_input = []
    for i  in range(len(round2_y_predictions)):
        
        if round2_y_predictions[i] == 1: #no feedback needed for that dimension. feedback potentially needed for next dinmension
            
            round3_X_train_input.append(round2_X_train_input[i])
            round3_y_train_input.append(y_thirdcol_train_vector[i])
    
    model3 = LogisticRegression(random_state = 0)
    model3 = model3.fit(round3_X_train_input, round3_y_train_input)
    round3_y_predictions = model3.predict(round3_X_train_input)
    
    classification_predictions = []
    
    print("input validation =", type(X_validate), " shape", X_validate.shape)
    classification_predictions = model1.predict(X_validate).reshape(-1,1)
    #print("model 1 predictions type = ", type(cola))
    classification_predictions = np.append(classification_predictions, model2.predict(X_validate).reshape(-1,1), axis = 1)
    classification_predictions = np.append(classification_predictions, model3.predict(X_validate).reshape(-1,1), axis = 1)
    
    
    
#     for given_response_vec in X_validate:
#         print("asking for prediction for = ", type(given_response_vec))
#         classifed = model1.predict(given_response_vec) + model2.predict(given_response_vec) + model3.predict(given_response_vec)
#         print("prediction should be list = ", type(classified))
#         classification_predictions.append(classified)
    
   #classification_predictions = np.array(classification_predictions)
    
    accuracy_l = []
    for i in range(3):
        dim_accuracy = metrics.accuracy_score(y_validate[:, i], classification_predictions[:,i])
        print("Accuracy is = ", dim_accuracy)
        accuracy_l.append(dim_accuracy)
        
        print("\n\nConfusion Matrix:")
        
        confusion_matrix = metrics.confusion_matrix(y_validate[:, i], classification_predictions[:,i], normalize="true")
        print(confusion_matrix)
        
        #uncomment next line to print the confusion matrix
        #plot_confusion_matrix(confusion_matrix, labels_order[i] + qname,  dim_accuracy)
        print("\n\n")
        
    return accuracy_l  #accuracy for the three dimensions for a given problem

In [188]:

import ast
        

for cur_qid in qids:
    
        
    cur_question_training_df = train_df[train_df["qid"] == cur_qid]
    cur_question_validation_df = validate_df[validate_df["qid"] == cur_qid]
    
    print("QID = \'{}.\'".format(cur_qid))
    qname_formatted = ": " + cur_qid #format cur_qid name so it looks nice when printed with other function
    
    q_goldenanswers = cur_question_training_df["example_correct_answers"].iloc[0]
    q_goldenanswers = ast.literal_eval(q_goldenanswers)
    
    if type(q_goldenanswers) != list:
        print("Had to convert to list")
        q_goldenanswers = [q_goldenanswers]

    
    cur_qid_accuracy = train_model_and_validate_specify_order(cur_question_training_df, cur_question_validation_df, 
                                                qname = qname_formatted, golden_answers = q_goldenanswers, 
                                                              given_dims_order = ["una","c", "hl"])
    
    qids_accuracy.append(cur_qid_accuracy)
    print("\n\n")


QID = 'print_found_if_three_numbers_unique.'
Shape of all student vectors = (96, 116)
Shape of golden_answers_vectors = (2, 116)
Shape single student vector = (116,)
Shape single golden answer vector = (116,)
After reshaping, current size = (1, 116) for golden answer and  (1, 116) for student vector
Shape single golden answer vector = (116,)
After reshaping, current size = (1, 116) for golden answer and  (1, 116) for student vector
Shape single student vector = (116,)
Shape single golden answer vector = (116,)
After reshaping, current size = (1, 116) for golden answer and  (1, 116) for student vector
Shape single golden answer vector = (116,)
After reshaping, current size = (1, 116) for golden answer and  (1, 116) for student vector
Shape single student vector = (116,)
Shape single golden answer vector = (116,)
After reshaping, current size = (1, 116) for golden answer and  (1, 116) for student vector
Shape single golden answer vector = (116,)
After reshaping, current size = (1, 116) f

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [ ]:
model1.predict(X_validate[:,1])
# a = np.array([15, 6, 7]).reshape(1,3)
# b = np.array([30, 12, 14]).reshape(1,3)

# cosine_similarity(a,b)

In [133]:
a = np.array([15, 6, 7]).reshape(1,-1)
b = np.array([30, 12, 14]).reshape(1,-1)

cosine_similarity(a,b)

0.9999999999999999

In [150]:
confusion_matrix = metrics.confusion_matrix([1], [2], normalize="true")

In [151]:
print(confusion_matrix)

[[0. 1.]
 [0. 0.]]


In [152]:
print(metrics.classification_report([2,3], [2,4]))

              precision    recall  f1-score   support

           2       1.00      1.00      1.00         1
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         0

    accuracy                           0.50         2
   macro avg       0.33      0.33      0.33         2
weighted avg       0.50      0.50      0.50         2



c:\users\chine\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\chine\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\chine\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 